In [11]:
import tkinter as tk
import sqlite3

# Initialize the SQLite database
conn = sqlite3.connect("lms.db")
cursor = conn.cursor()

# Create the tables if they don't exist
cursor.execute('''
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT UNIQUE,
        password TEXT
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS courses (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        title TEXT,
        description TEXT
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS progress (
        user_id INTEGER,
        course_id INTEGER,
        completed BOOLEAN,
        FOREIGN KEY (user_id) REFERENCES users(id),
        FOREIGN KEY (course_id) REFERENCES courses(id)
    )
''')

# Create the tkinter window
root = tk.Tk()
root.title("Learning Management System")

root.configure(bg="#5D6D7E")

width= root.winfo_screenwidth() 
height= root.winfo_screenheight()
#setting tkinter window size
root.geometry("%dx%d" % (width, height))

# Function to add a new course
def add_course():
    title = course_title.get()
    description = course_description.get()
    cursor.execute("INSERT INTO courses (title, description) VALUES (?, ?)", (title, description))
    conn.commit()
    course_title.delete(0, tk.END)
    course_description.delete(0, tk.END)
    show_courses()

# Function to show available courses
def show_courses():
    main_menu_frame.pack_forget()
    main_menu_frame.pack()
    courses_list.delete(0, tk.END)
    for row in cursor.execute("SELECT title FROM courses"):
        courses_list.insert(tk.END, row[0])

# Function to enroll in a course
def enroll_course():
    selected_course = courses_list.get(courses_list.curselection())
    course_id = cursor.execute("SELECT id FROM courses WHERE title=?", (selected_course,)).fetchone()[0]
    cursor.execute("INSERT INTO progress (user_id, course_id, completed) VALUES (?, ?, ?)", (current_user_id, course_id, False))
    conn.commit()
    show_progress()  # Refresh the list of courses after enrolling

# Function to show enrolled courses and track progress
def show_progress():
     
    progress_list.delete(0, tk.END)
    for row in cursor.execute("SELECT courses.title, progress.completed FROM courses JOIN progress ON courses.id = progress.course_id WHERE progress.user_id=?", (current_user_id,)):
        progress_list.insert(tk.END, f"Course: {row[0]}, Completed: {'Yes' if row[1] else 'No'}")
      

# Function to mark a course as completed
def mark_completed():
    selected_progress = progress_list.get(progress_list.curselection())
    course_title = selected_progress.split("Course: ")[1].split(", Completed:")[0].strip()
    
    # Query the database for the course_id
    cursor.execute("SELECT id FROM courses WHERE title=?", (course_title,))
    course_id = cursor.fetchone()
    
    if course_id:
        course_id = course_id[0]
        cursor.execute("UPDATE progress SET completed = ? WHERE user_id = ? AND course_id = ?", (True, current_user_id, course_id))
        conn.commit()
        show_progress()
    else:
        print("Course not found in the database.")


# Function for user login
def login():
    username = username_entry.get()
    password = password_entry.get()
    user_data = cursor.execute("SELECT id, username, password FROM users WHERE username=? AND password=?", (username, password)).fetchone()
    
    if user_data:
        global current_user_id
        current_user_id = user_data[0]
        login_frame.pack_forget()
        main_menu_frame.pack()
        show_courses()
        show_progress()
    

# Function to switch to the sign-up frame
def switch_to_signup_frame():
    signup_frame.pack()
    login_frame.pack_forget()
def switch_to_login_frame():
    signup_frame.pack_forget()
    login_frame.pack()

# Function to create a new user
def create_account():
    username = signup_username_entry.get()
    password = signup_password_entry.get()
    cursor.execute("INSERT INTO users (username, password) VALUES (?, ?)", (username, password))
    conn.commit()
    signup_username_entry.delete(0, tk.END)
    signup_password_entry.delete(0, tk.END)
    switch_to_login_frame()

# Create and configure frames
login_frame = tk.Frame(root, bg="#2E4053", bd=20, highlightbackground="#FFC300", highlightthickness=2, height=500)
login_frame.pack(pady=150)

signup_frame = tk.Frame(root, bg="#2E4053", bd=20, highlightbackground="#FFC300", highlightthickness=2, height=500)
signup_frame.pack_forget()

main_menu_frame = tk.Frame(root,bg="#2E4053", bd=20, highlightbackground="#FFC300", highlightthickness=2)

# Create and configure widgets
login_label = tk.Label(login_frame, text="Login", font=("Times", "50", "bold italic"), bg="#2E4053")
username_label = tk.Label(login_frame, text="Username:", font=("Times", "20"), bg="#2E4053")
password_label = tk.Label(login_frame, text="Password:", font=("Times", "20"), bg="#2E4053")
username_entry = tk.Entry(login_frame, width=40)
password_entry = tk.Entry(login_frame, show="*", width=40)
login_button = tk.Button(login_frame, text="Login", command=login, bg="#DAF7A6")
create_account_button = tk.Button(login_frame, text="Create Account", command=switch_to_signup_frame, bg="#DAF7A6")

signup_label = tk.Label(signup_frame, text="Create Account", font=("Times", "50", "bold italic"), bg="#2E4053")
signup_username_label = tk.Label(signup_frame, text="Username:", font=("Times", "20"), bg="#2E4053")
signup_password_label = tk.Label(signup_frame, text="Password:", font=("Times", "20"), bg="#2E4053")
signup_username_entry = tk.Entry(signup_frame, width=40)
signup_password_entry = tk.Entry(signup_frame, show="*", width=40)
signup_button = tk.Button(signup_frame, text="Create Account", command=create_account, bg="#DAF7A6")


wel=tk.Label(main_menu_frame,text="Welcome ",font=("Times", 20, "bold italic"), bg="#2E4053")
course_label = tk.Label(main_menu_frame, text="Add a New Course",font=('Times',14), bg="#2E4053")
course_title = tk.Entry(main_menu_frame, width=30)
course_description = tk.Entry(main_menu_frame, width=30)
add_course_button = tk.Button(main_menu_frame, text="Add Course", command=add_course, bg="#DAF7A6")
courses_list = tk.Listbox(main_menu_frame, selectmode=tk.SINGLE, width=50)
enroll_button = tk.Button(main_menu_frame, text="Enroll in Course", command=enroll_course, bg="#DAF7A6")

progress_label = tk.Label(main_menu_frame, text="Course Progress",font=('Times',14), bg="#2E4053")
progress_list = tk.Listbox(main_menu_frame, selectmode=tk.SINGLE, width=50)
mark_completed_button = tk.Button(main_menu_frame, text="Mark as Completed", command=mark_completed, bg="#DAF7A6")

# Place widgets on the frames
login_label.pack()
username_label.pack()
username_entry.pack()
password_label.pack()
password_entry.pack()
login_button.pack()
create_account_button.pack()

signup_label.pack()
signup_username_label.pack()
signup_username_entry.pack()
signup_password_label.pack()
signup_password_entry.pack()
signup_button.pack()

wel.pack()
course_label.pack()
course_title.pack()
course_description.pack()
add_course_button.pack()
courses_list.pack()
enroll_button.pack()

progress_label.pack()
progress_list.pack()
mark_completed_button.pack()

main_menu_frame.pack_forget()

# Start the tkinter main loop
root.mainloop()

# Close the database connection
conn.close()